In [2]:
%cd ..


/Users/gianlucascarpellini/dev/vipm-project


In [8]:
import torch
from matcher.features import FeatureMatcher
from torch.utils.data import DataLoader
from matcher.dataset import ClassificationDataset
from torchvision import transforms

config = {
    'data_path': 'data/fashion-product-images-small/images',
    'exp_base_dir': 'data/exps/exp1',
    'image_size': (224, 224),
    'load_path': "data/models/resnet18_best.pt",
    'features_path': 'data/features/featuresresnet18.npy',
    'index_path': 'data/features/featuresresnet18_index.pickle',
    'segmentation_path': 'data/models/segm.pth',
    "classes": ["masterCategory", "subCategory", "gender"],
    "segmentation":False

}
# ["gender", "subCategory", "masterCategory"]


In [9]:
fm = FeatureMatcher(features_path=config['features_path'], model_path=config['load_path'],
                    index_path=config['index_path'],
                    segmentation_model_path=config['segmentation_path'])



Loading model
Loaded in 33.04561901092529s


In [14]:
print("Loading")
test_loader = DataLoader(
    ClassificationDataset(
        "./data/fashion-product-images-small/images",
        "./data/small_test.csv",
        distinguish_class=config["classes"],
        image_size=config["image_size"],
        transform=transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    ),
        thr=5,
    ),
    batch_size=1,
    shuffle=True,
    )

Loading
     masterCategory subCategory gender
0          Footwear       Shoes    Men
1       Accessories       Belts  Women
2       Accessories        Bags  Women
3       Accessories        Bags  Women
4          Footwear      Sandal    Men
...             ...         ...    ...
8880    Accessories       Belts    Men
8881    Accessories     Watches  Women
8882    Accessories     Watches    Men
8883    Accessories       Belts  Women
8884       Footwear       Shoes    Men

[8819 rows x 3 columns]
       masterCategory subCategory gender
count            8819        8819   8819
unique              7          39      5
top           Apparel     Topwear    Men
freq             4239        3082   4436
masterCategory  subCategory       gender
Accessories     Accessories       Men        20
                                  Unisex      8
                Bags              Men        19
                                  Unisex    204
                                  Women     422
             

In [20]:
device = "cpu"
with torch.no_grad():
    accurate_labels = 0
    all_labels = 0
    val_loss = []
    accurate_labels = [0, 0, 0]
    accuracies = [0, 0, 0]
    for batch_idx, (data, target) in enumerate(test_loader):
        for i in range(len(data)):
            data[i] = data[i].to(device)
            
        for similarity in range(0, 4): # Per ogni classe di similarità
            if config["segmentation"]:
                image = fm.segment_image(image)
            output = fm.get_k_most_similar(data[0], similar_type=similarity,
                                           image_size=config["image_size"])
            target = target.long()
    

        all_labels += len(target)


RuntimeError: requested resize to (224, 224) ((224, 224) elements in total), but the given tensor has a size of 3x224x224 (150528 elements). autograd's resize can only change the shape of a given tensor, while preserving the number of elements. 

In [ ]:
    for i in range(n_label):
        accuracies[i] = 100.0 * accurate_labels[i].item() / all_labels
    print(
        "Test accuracy: ({})/{} ({}), Loss: ({})".format(
            ", ".join([str(accurate_labels[i].item()) for i in range(n_label)]),
            all_labels,
            ", ".join(["{:.3f}%".format(accuracies[i]) for i in range(n_label)]),
            ", ".join(
                "{:.6f}".format(loss)
                for loss in torch.mean(torch.tensor(val_loss), dim=0).data.tolist()
            ),
        )
    )

image = Image.open(local_filename).convert("RGB")
if segmentation:
    image = fm.segment_image(image)
classes = fm.classify(image, image_size=image_size)

labels = [f"In base al tuo sesso {classes[0]}",
          f"In base al tipo di abito {classes[1]}",
          f"In base al sottotipo di abito {classes[2]}",
          f"Simili al tuo stile"]

for i in range(0, 4):
    bot.sendMessage(chat_id, labels[i])
    result = fm.get_k_most_similar(image, image_size=image_size,
                                   k=k, similar_type=i)